In [99]:
import sys
sys.path.append("/tmp")
from imp import reload
import article_loader
reload(article_loader)

dirname = "/var/pti/scrape"
loader = article_loader.ArticleLoader()

(idx2doc, dataset) = loader.load_articles(dirname)

In [101]:
print(idx2doc)
print(datasets)

['18', '9', '24', '23', '7', '26', '19', '13', '10', '21', '11', '22', '1', '25', '6', '12', '15', '2', '20', '27', '8', '16', '14', '17']
['\u3000 神奈川県小田原市 で 生活保護受給者 の 自立支援 を 担当 する 市職員 ら が 「 不正受給 は クズ だ 」 と の 趣旨 の 英文 が 書か れ た ジャンパー を 着 て い た 問題 で 、 職員 ら が 「 生活保護 悪 撲滅 チーム 」 を 示す 「 ＳＨＡＴ 」 の マーク を 袖 に 付け た 夏 用 の ポロシャツ を 製作 し 、 業務 で 着用 し て い た こと が ３ 日 、 市 へ の 取材 で 分かっ た 。 \u3000 市 は 不適切 な マーク だ として 、 同日 以降 の 使用 を 禁止 し た 。 \u3000 市 に よる と 、 ポロシャツ は 、 ジャンパー が 作ら れ た ２ ０ ０ ７ 年 の 翌年 夏 に 製作 さ れ 、 現在 も ２ ８ 人 の 職員 が 所持 し て いる 。 青 や 紺 など 数種類 あり 、 １ 着 ２ 千 円 前後 で 各 職員 が 個人 として 購入 し て い た 。  トランプ大統領 の 円安 誘導 批判 は 実力行使 を 伴わ ぬ 空砲 で あり 、 神通力 を 失う の は 時間 の 問題 だ と 野村証券 の 池田雄之輔 氏 は 分析 。 \xa0 記事 の 全文 ドル円 相場 波乱 の 芽 は 、 中期的 に は トランプ政権 の 動向 より も 金融政策 に 潜ん で いる と バークレイズ 証券 の 門田 真一郎 氏 は 指摘 。 \xa0 記事 の 全文 トムソン・ロイター は 世界最大 級 の 国際 マルチメディア 通信社 です 。 マーケット ニュース 、 ビジネスニュース 、 テクノロジー ニュース 、 外国為替 フォーラム 、 ワールド 、 ヘッドラインニュース 、 速報 ニュース 、 コラム 、 ブログ 、 国内 株式 、 海外 株式 、 外国為替 、 投資信託 情報 が 、 ロイター . co.jp サイト 、 ビデオ 、 モバイルサイト 、 インタラクテ

In [79]:
from imp import reload
import pti_lda
reload(pti_lda)

<module 'pti_lda' from '/tmp/pti_lda.py'>

In [80]:
lda = pti_lda.PTILDA()

In [81]:
lda.train(dataset)


In [84]:
lda.dump_topic(idx2doc)

In [82]:
lda.print_topic_word(20)

Topic #0:
ロイター ます ニュース 情報 株式 職員 トムソン 日本 10分 ポロシャツ 関連 金融 会談 マティス ご覧 問題 あり 記事 いる 負担

Topic #1:
ます 共同記者会見 不適切 選挙中 株式 全文 職員 ニュース 情報 デー トランプ大統領 取材 トムソン 経費 相場 関連 作ら おり ジャンパー トランプ



In [83]:
lda.print_doc_topic(1)

Doc 0:
Topic:0, prob:0.9953174983462851

Doc 1:
Topic:0, prob:0.9860654820955974

